# CuPy

* CuPy is functionally similar to Numba-CUDA but more closely mimics NumPy
* Many operations can be written using NumPy-like syntax
* CuPy has drop-in equivalents of most NumPy operations and syntax syntax with GPU arrays (`cp.ndarray`), ufuncs, broadcasting, reductions.
* Unlike Numba-CUDA, it provides implementations of a number of more complex operations in `cupyx.scipy`
    * FFT
    * linear algebra
    * sparse matrices
    * signal processing
    * random numbers
* These can make cupy easier to use for many applications


## CuPy Array Basics (NumPy, but on GPU)
* Basically, you can replace `numpy as np` with `cupy as cp` and most ufuncs and array ops just work


In [ ]:

import cupy as cp
a = cp.arange(10**6, dtype=cp.float32)
b = cp.sin(a) + cp.cos(a)  # ufuncs on GPU
print(b.shape, b.dtype, type(b))
print("Host preview:", cp.asnumpy(b[:5]))


(1000000,) float32 <class 'cupy.ndarray'>
Host preview: [ 1.          1.3817732   0.49315065 -0.8488725  -1.4104462 ]


* Using `cp.arange` will allocate an array on GPU device memory
* Ufuncs will perform operations on the GPU
* `cp.asnumpy` will copy back to the host
* To minimize data transfer, use operations like `cp.asarray` and `cp.asnumpy` only when needed

### Broadcasting, reductions, and axis ops
* The normal broadcasting, indexing, and reduction operations operate as one would expect

In [ ]:

x = cp.random.random((4096, 4096), dtype=cp.float32)
w = cp.linspace(0, 1, x.shape[1], dtype=cp.float32)
y = x * w                       # broadcast
col_means = y.mean(axis=0)      # reduction on GPU
print(col_means.shape, col_means.dtype)


(4096,) float32



## Defining Kernels

* There are a handful of options for defining kernels in CuPy. These present many options for balancing complexity and fine-grain control.

| Approach | Level of Control | Language you write | How you define it | How you launch it | Can use `threadIdx`/`blockIdx` & shared mem? | Best for | Pros | Cons |
|---|---|---|---|---|---|---|---|---|
| **Vectorized CuPy ops (ufuncs, broadcasting, reductions)** | Low | Python (NumPy-like) | Just write `cp.sin(x)`, `x*y`, `x.mean(axis=0)`, etc. | Regular function call | No, abstracted away | Elementwise/reduction math, linear algebra via `cp.linalg`, FFT via `cupyx.scipy.fft` | Fast to write; leverages vendor libs (cuBLAS/cuFFT); very readable | Limited low-level control; performance tuning mostly indirect |
| **`ElementwiseKernel`** | Medium | CUDA C *expression strings* | `ElementwiseKernel(in_params, out_params, operation, name)` | Call like a function: `k(x, ...)` | no explicit thread/block math | Custom elementwise transforms | Minimal boilerplate; in-place and broadcasting friendly | Logic must fit elementwise form; no shared memory |
| **`ReductionKernel`** | Medium | CUDA C *expression strings* | `ReductionKernel(in_params, out_params, map_expr, reduce_expr, post_map_expr, identity, name)` | Call like a function: `k(x, ...)` | no | Custom reductions (sum of f(x), argmax, etc.) | Concise custom reductions; good perf | Reduction pattern only; more complex to reason about |
| **`cupyx.jit` – `@jit.kernel()`** | Medium–High | Restricted Python (JIT to CUDA) | Write Python function with array indexing; decorate with `@jit.kernel()` | Call like a function (grid/block inferred or provided) | APIs for `threadIdx`, `blockIdx`, shared mem via `jit.shared_memory`) | Custom elementwise/nd kernels with Python syntax | Pythonic; no C string; easy to iterate | Still a restricted subset; not as feature-complete as raw CUDA C |
| **`cupyx.jit` – `@jit.rawkernel()`** | High | Restricted Python with explicit launch math | Define func with `@jit.rawkernel()` and compute global index manually | CUDA-like launch: `f[(blocks,), (threads,)](args...)` |full index control; shared mem via `jit.shared_memory` | Hand-tuned kernels without leaving Python | Good balance of control & ergonomics | Some CUDA features may require workarounds; subset semantics |
| **`RawKernel`** | Highest | CUDA C/C++ (as a string) | `cp.RawKernel(src, "func_name")` with CUDA code | CUDA-like launch: `k((blocks,), (threads,), (args...), shared_mem=...)` | full control | Performance-critical kernels; shared memory tiling; intrinsics | Full CUDA power; predictable | You write/maintain CUDA C strings; harder to prototype |
| **`RawModule`** | Highest | CUDA C/C++ (string or file), PTX | `cp.RawModule(code=..., path=..., options=...)`; then `get_function("name")` | Same as `RawKernel` once you get the function | yes| Packaging multiple kernels, using headers, templates, linking | Organize many kernels; reuse across cells | Build/link errors are more complex; still CUDA C |

* The best way to start out with CuPy is to use the vectorized operations, ufuncs, broadcasting / other NumPy-esque features
* Branch out and expand whre more fine-grain control / complexity is needed

> **Rule of thumb:** Start with **CuPy** vectorization; drop to `cupyx.jit`/RawKernel or **Numba** only for hotspots that need custom kernels.



## Elementwise Kernels
* Elemntwise and reduction kernels can be a convenient alternative to writing full CUDA kerneels
* You don't have to worry about launch syntax like specifying the number of blocks / threads per block


In [ ]:

from cupy import ElementwiseKernel, ReductionKernel
import cupy as cp

# Elementwise: y = a*x + b
axpb = ElementwiseKernel(
    in_params='float32 x, float32 a, float32 b',
    out_params='float32 y',
    operation='y = a * x + b;',
    name='axpb'
)

x = cp.linspace(0, 1, 8, dtype=cp.float32)
y = axpb(x, 2.0, 1.0)
print("Elementwise:", y)

Elementwise: [1.        1.2857143 1.5714285 1.8571429 2.142857  2.4285715 2.7142859
 3.       ]


## Reduction Kernels

* The `cupy.ReductionKernel` API lets you define your own reductions on the GPU (e.g. sum, mean, norm, min/max, etc.) using a few simple expressions — no need to write explicit CUDA kernels.

* It's roughly analogous to a CUDA kernel that:
1. Maps each input element through some expression (`map_expr`),
2. Reduces all mapped results using an associative binary operation (`reduce_expr`),
3. Optionally transforms the final accumulator before writing output (`post_map_expr`),
4. Starts with an identity (neutral) value for the accumulator.


* Here's an example of a reduction kernel

In [ ]:

# Reduction: sum of squares
sum_squares = ReductionKernel(
    in_params='float32 x',
    out_params='float32 y',
    map_expr='x * x',
    reduce_expr='a + b',
    post_map_expr='y = a',
    identity='0',
    name='sum_squares'
)
print("Reduction:", sum_squares(cp.arange(10, dtype=cp.float32)))


Reduction: 285.0


## Raw Kernels
* A raw kernel is basically justa CUDA C kernel written as a string
* `cp.RawKernel` lets you write CUDA C kernels (as strings) and launch from Python
* This is useful for relatively simple kernels that might not be easily expressed as NumPy-like vectorized operations
* Raw kernels require you to specify the block and thread count when launching as in Numba-CUDA / standard CUDA kernels

In [ ]:
import cupy as cp

# RawKernel (CUDA C)
saxpy_src = r'''
extern "C" __global__
void saxpy(const float a, const float* __restrict__ x,
           const float* __restrict__ y, float* __restrict__ out, int n) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < n) out[i] = a * x[i] + y[i];
}
''';
saxpy_kernel = cp.RawKernel(saxpy_src, "saxpy")

# test data
n = 1_000_000
x = cp.random.random(n, dtype=cp.float32)
y = cp.random.random(n, dtype=cp.float32)
out = cp.empty_like(x)

threads = 256
blocks = (n + threads - 1) // threads
saxpy_kernel((blocks,), (threads,), (2.0, x, y, out, n))
cp.cuda.Stream.null.synchronize()
print("RawKernel ok; out[0:5] =", out[:5])


RawKernel ok; out[0:5] = [0.29604462 0.523803   0.5467715  0.13893062 0.36670983]


> **Note** For a raw kernel, we want to use the `synchronize` operation like in Numba-CUDA to ensure that all threads have finished before we need to use the output of the kernel. CuPy uses the default stream, called the null stream for computations by default.


## `cupyx.jit`: CUDA-like kernels in Python
* `cupyx.jit` JIT-compiles a restricted Python subset to CUDA
* Its a middle ground between high-level array ops and CUDA C strings
* Very similar to defining Numba-CUDA kernels defined in standard python


In [ ]:

from cupyx import jit
import cupy as cp

@jit.rawkernel()
def saxpy_jit(a, x, y, out):
    i = jit.blockDim.x * jit.blockIdx.x + jit.threadIdx.x
    if i < x.size:
        out[i] = a * x[i] + y[i]

n = 1_000_000
x = cp.random.random(n, dtype=cp.float32)
y = cp.random.random(n, dtype=cp.float32)
out = cp.empty_like(x)
threads = 256
blocks = (n + threads - 1) // threads
saxpy_jit[(blocks,), (threads,)](2.0, x, y, out)
cp.cuda.Stream.null.synchronize()
print("cupyx.jit ok; out[0:5] =", out[:5])


/usr/local/lib/python3.12/dist-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


cupyx.jit ok; out[0:5] = [1.6782546  1.2052308  1.0658538  0.5613324  0.48698527]



## SciPy‑Compatible: FFT, Linear Algebra, Sparse
* CuPy has existing kernels for solving linear algebra problems, performing FFTs and many other common tasks
* `cupyx.scipy` mirrors SciPy APIs and calls high-performance GPU libs (cuBLAS, cuFFT, cuSPARSE, etc.).

In [ ]:
import cupy as cp
from cupyx.scipy import fftpack, linalg, sparse

# FFT round‑trip
x = cp.random.random(1<<20).astype(cp.complex64)
X = fftpack.fft(x)
x_rec = fftpack.ifft(X)
print("FFT round‑trip error:", float(cp.max(cp.abs(x - x_rec))))

# Linear solve: Ax=b
A = cp.random.random((1024, 1024), dtype=cp.float32)
b = cp.random.random(1024, dtype=cp.float32)
x = cp.linalg.solve(A, b)
r = cp.linalg.norm(A @ x - b) / cp.linalg.norm(b)
print("Solve relative residual:", float(r))

# Sparse example
rows = cp.array([0, 0, 1, 2, 2, 2])
cols = cp.array([0, 2, 2, 0, 1, 2])
vals = cp.array([1, 2, 3, 4, 5, 6], dtype=cp.float32)
S = sparse.coo_matrix((vals, (rows, cols)), shape=(3,3)).tocsr()
d = S.dot(cp.array([1,2,3], dtype=cp.float32))
print("Sparse dot:", d)

FFT round‑trip error: 7.802258892297687e-07
Solve relative residual: 0.00032664884929545224
Sparse dot: [ 7.  9. 32.]



## Random Numbers
* Random numbers are a bit nicer to handle in CuPy than Numba-CUDA


In [ ]:
import cupy as cp
rng = cp.random.default_rng(seed=123)
u = rng.random(5, dtype=cp.float32)
n = rng.standard_normal(5, dtype=cp.float32)
print("Uniform:", u)
print("Normal :", n)

Uniform: [0.9656452  0.64809096 0.61324495 0.18948276 0.69727665]
Normal : [-0.6466216   0.54218817 -1.4606272  -0.6613516  -0.35003424]


In [ ]:

# CUDA events
import cupy as cp
start, stop = cp.cuda.Event(), cp.cuda.Event()

x = cp.random.random((1<<24,), dtype=cp.float32)
start.record()
y = cp.tanh(cp.sin(x) + 0.1*x)
stop.record(); stop.synchronize()
print("Elapsed (ms):", cp.cuda.get_elapsed_time(start, stop))



## Streams in CuPy
* Recall that streams are independent queues of GPU work.
* Operation in the same stream execute in order
* Operations in different streams may run concurrently
* By default, CuPy uses the **null (default) stream**. You can create additional streams to overlap kernels and transfers.

### Key ideas
- **Default vs custom streams**  
  - Default stream: `cp.cuda.Stream.null` (used unless you set your own).
  - Custom stream: `s = cp.cuda.Stream(non_blocking=True)`; use with a context manager.
- **Asynchrony**  
  - Kernel launches and most copies are asynchronous with respect to the host (CPU).
  - Use `stream.synchronize()` or `cp.cuda.Stream.null.synchronize()` when you need results or accurate timings.

In [ ]:
import cupy as cp
import math, time


N = 50_000_000
x = cp.random.random(N, dtype=cp.float32)
y = cp.random.random(N, dtype=cp.float32)

def heavy_op(a):
    # A chain of elementwise ops to keep the GPU busy
    return cp.tanh(cp.sin(a) + 0.1*a) + cp.sqrt(cp.abs(a))

### Single stream computation
r1 = heavy_op(x)
r2 = heavy_op(y)
cp.cuda.Stream.null.synchronize()

### Multiple stream computation
s1 = cp.cuda.Stream(non_blocking=True)
s2 = cp.cuda.Stream(non_blocking=True)

with s1:
    r1_ov = heavy_op(x)
with s2:
    r2_ov = heavy_op(y)

s1.synchronize()
s2.synchronize()

* As in Numba-CUDA, you may be able to speed up your computation by using overlapping transfer and compute with a Ping-Pong kernel or other multi-stream methods

## CuPy's Memory Pool

* GPU allocations via the CUDA driver (`cudaMalloc` / `cudaFree`) are expensive operations compared to CPU `malloc/free`.
* They cross the user-kernel boundary, update GPU virtual memory structures, and often imply device synchronization.
* In tight loops with many small arrays, this overhead can dominate runtime.
* CuPy's memory pool acts like a caching allocator:
  * New CuPy arrays are served from a pool of already-allocated GPU blocks, avoiding repeated `cudaMalloc` calls.
  * When arrays die, memory returns to the pool (not immediately to the driver), so future allocations can reuse it instantly.

### Key takeaways
- `cudaMalloc`/`cudaFree` are heavyweight; frequent calls serialize work and stall the GPU.
- The memory pool removes most allocation overhead by reusing blocks.
- You can inspect/reset the pool:

In [ ]:
mpool = cp.get_default_memory_pool()
print("pool total bytes:", mpool.total_bytes())
print("pool used bytes:", mpool.used_bytes())
mpool.free_all_blocks()  # return cached blocks to the driver

pool total bytes: 1279109632
pool used bytes: 1248805888



* As an example, let's see what happens when we do raw `cudaAlloc` and `cudaFree` versus relying ont the memory pool

In [ ]:

import cupy as cp, time

def alloc_free_loop(n=1000, shape=(1024, 1024), dtype=cp.float32):
    start = time.perf_counter()
    for _ in range(n):
        x = cp.empty(shape, dtype=dtype)
        # Do nothing with x; we are measuring allocation + free path
        del x
    cp.cuda.Stream.null.synchronize()
    return time.perf_counter() - start

#  Disable CuPy's memory pool (use raw cudaMalloc/cudaFree)
cp.cuda.set_allocator(None)  # turn off pooled allocator
t_no_pool = alloc_free_loop()

# Re-enable CuPy's default memory pool
mpool = cp.cuda.MemoryPool()
cp.cuda.set_allocator(mpool.malloc)
t_pool = alloc_free_loop()

print(f"Without pool: {t_no_pool:.3f}s")
print(f"With pool:    {t_pool:.3f}s  (speedup: {t_no_pool/t_pool:.1f}×)")

Without pool: 0.143s
With pool:    0.005s  (speedup: 28.9×)


## Pinned Memory
* As in Numba-CUDA, CuPy allows you to use pinned memory. Recall that pinned memory is page locked memory that the OS will not page out which allows higher host to device bandwidth
* Pinned memory allows for true asynchronous copies that overlap with kernels on other streams
* In contrast, copies from pageable memory may inovle an internal pin to copy to unpin staging path that blocks on the host, preventing work from being queued on another stream


### When pinned memory helps
* To reiterate a few points on pinned memory, pinned memory can be useful when:
  - You frequently transfer large arrays between host and device.
  - You want to overlap copies with compute using multiple CUDA streams.

>  Pinned memory is a limited system resource. Over-pinning large regions can hurt overall system performance. Reuse pinned buffers; don't pin “just in case.”


### Using pinned memory in CuPy
* You can use pinned memory like this:


In [ ]:
import numpy as np, cupy as cp

n = 10_000_000
buf = cp.cuda.alloc_pinned_memory(n * np.dtype(np.float32).itemsize)
h = np.frombuffer(buf, dtype=np.float32, count=n)  # NumPy view, no copy
h.fill(1.0)  # normal NumPy ops
# Copy to device memory from pinned memory
d = cp.asarray(h)

* By default, CuPy uses a pinned memory pool similar to the normal memory pool

In [ ]:
mpool = cp.get_default_memory_pool()

In [ ]:
import cupy as cp

pinned_pool = cp.cuda.PinnedMemoryPool()
cp.cuda.set_pinned_memory_allocator(pinned_pool.malloc)


buf = cp.cuda.alloc_pinned_memory(n * np.dtype(np.float32).itemsize)
print("free blocks:", pinned_pool.n_free_blocks())

# Now any cp.cuda.alloc_pinned_memory(...) call will use this pool.
# Also, if CuPy internally needs pinned buffers, it can reuse from here.


# Return cached pinned blocks to the OS if you want to trim:
# pinned_pool.free_all_blocks()

free blocks: 0


### Benchmark

* We can compare how long it takes to copy data from pageable versus pinned memory to device memory to see the performance benefit
* The basic idea here is that we allocate an array `d` on the GPU. We'll repeatedly do some copies from pageable versus pinned memory to `d` to see the difference in performance

In [4]:
import cupy as cp
import numpy as np

# Ensure a pinned pool is installed (helps reuse pinned buffers)
pinned_pool = cp.cuda.PinnedMemoryPool()
cp.cuda.set_pinned_memory_allocator(pinned_pool.malloc)

N = 50_000_000
bytes_ = N * np.dtype(np.float32).itemsize

# Pageable host array (regular NumPy)
h_pageable = np.ones(N, dtype=np.float32)

# Pinned host array
buf = cp.cuda.alloc_pinned_memory(bytes_)
h_pinned = np.frombuffer(buf, dtype=np.float32, count=N)
h_pinned.fill(1.0)

d = cp.empty(N, dtype=cp.float32)

def time_h2d(src, repeats=3):
    times = []
    for _ in range(repeats):
        start, stop = cp.cuda.Event(), cp.cuda.Event()
        start.record()
        d.set(src)
        stop.record()
        stop.synchronize()

        times.append(cp.cuda.get_elapsed_time(start, stop))

    return np.array(times).mean()

# H2D pageable (default stream)
t_pageable = time_h2d(h_pageable)

# H2D pinned
t_pinned = time_h2d(h_pinned)

print(f"H2D pageable (ms): {t_pageable:.1f}")
print(f"H2D pinned   (ms): {t_pinned:.1f}")
print(f"Speedup: {t_pageable / t_pinned:.2f}×")

H2D pageable (ms): 42.4
H2D pinned   (ms): 16.2
Speedup: 2.62×



## References
- CuPy docs: https://docs.cupy.dev  
- Numba CUDA: https://numba.readthedocs.io/en/stable/cuda/index.html  
- CUDA Array Interface: https://numba.readthedocs.io/en/stable/cuda/cuda_array_interface.html  
- DLPack spec: https://dmlc.github.io/dlpack/latest/
